In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [2]:
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", None)

In [ ]:
enrol_df = pd.read_csv(r"D:\UIDAI hackathon\North\UP\enrollment.csv")
demo_df = pd.read_csv(r"D:\UIDAI hackathon\North\UP\demographic.csv")
bio_df = pd.read_csv(r"D:\UIDAI hackathon\North\UP\biometric.csv")

In [4]:
all_dfs = [enrol_df, demo_df, bio_df]

In [5]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,01-04-2025,Uttar Pradesh,Maharajganj,273305,141,176,13
1,01-04-2025,Uttar Pradesh,Banda,210001,250,177,30
2,01-04-2025,Uttar Pradesh,Lucknow,226005,83,92,23
3,01-04-2025,Uttar Pradesh,Kanpur Nagar,208004,73,44,16
4,01-04-2025,Uttar Pradesh,Bahraich,271824,68,58,11


In [6]:
enrol_df["date"] = pd.to_datetime(enrol_df["date"], dayfirst=True)
demo_df["date"] = pd.to_datetime(demo_df["date"], dayfirst=True)
bio_df["date"] = pd.to_datetime(bio_df["date"], dayfirst=True)

In [7]:
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

89 89 95


In [8]:
enrol_df["district"].value_counts().sort_index()

district
Agra                          2357
Aligarh                       2463
Allahabad                     3116
Ambedkar Nagar                2088
Amethi                        2170
Amroha                         975
Auraiya                       1064
Ayodhya                        779
Azamgarh                      3526
Baghpat                       1137
Bagpat                           4
Bahraich                      1864
Ballia                        2718
Balrampur                     1136
Banda                          885
Bara Banki                    2512
Barabanki                      420
Bareilly                      1887
Basti                         1930
Bhadohi                        444
Bijnor                        2078
Budaun                        1677
Bulandshahar                   172
Bulandshahr                   2633
Chandauli                     1292
Chitrakoot                     587
Deoria                        2034
Etah                          1016
Etawah     

In [9]:
bio_df["district"].value_counts().sort_index()

district
Agra                           6574
Aligarh                        8231
Allahabad                      9953
Ambedkar Nagar                 7171
Ambedkar Nagar *                  1
Amethi                         8557
Amroha                         3075
Auraiya                        3881
Auraiya *                         1
Ayodhya                        7513
Azamgarh                      11236
Baghpat                        4084
Baghpat *                         3
Bagpat                           42
Bahraich                       5345
Ballia                         8845
Balrampur                      3299
Banda                          2906
Bara Banki                     8847
Barabanki                      3139
Bareilly                       5942
Basti                          6436
Bhadohi                        2887
Bijnor                         6313
Budaun                         5754
Bulandshahar                   1102
Bulandshahr                   10026
Chandauli          

In [10]:
demo_df["district"].value_counts().sort_index()

district
Agra                          2916
Aligarh                       3686
Allahabad                     4280
Ambedkar Nagar                3116
Amethi                        3587
Amroha                        1300
Auraiya                       1716
Ayodhya                       3067
Azamgarh                      4904
Baghpat                       1789
Baghpat *                        2
Bagpat                          16
Bahraich                      2313
Ballia                        3974
Balrampur                     1445
Banda                         1212
Bara Banki                    3819
Barabanki                     1309
Bareilly                      2560
Basti                         2776
Bhadohi                       1226
Bijnor                        2794
Budaun                        2519
Bulandshahar                   465
Bulandshahr                   4370
Chandauli                     1922
Chandauli *                      1
Chitrakoot                     766
Chitrakoot 

In [11]:
cleanup_map = {
    "Ambedkar Nagar *": "Ambedkar Nagar",
    "Gautam Buddh Nagar": "Gautam Buddha Nagar",
    "Auraiya *": "Auraiya",
    "Chandauli *": "Chandauli",
    "Chitrakoot *": "Chitrakoot",
    "Gautam Buddha Nagar *": "Gautam Buddh Nagar",
    "Jyotiba Phule Nagar *": "Amroha",
    "Mahoba *": "Mahoba",
    "Sant Kabir Nagar *": "Sant Kabir Nagar",
    "Baghpat *": "Baghpat",
    "Chandauli *": "Chandauli",
    "Allahabad": "Prayagraj",
    "Faizabad": "Ayodhya",
    "Jyotiba Phule Nagar": "Amroha",
    "Bara Banki": "Barabanki",
    "Bulandshahar": "Bulandshahr",
    "Kushi Nagar": "Kushinagar",
    "Kushinagar *": "Kushinagar",
    "Rae Bareli": "Raebareli",
    "Siddharth Nagar": "Siddharthnagar",
    "Shravasti": "Shrawasti",
    "Mahrajganj": "Maharajganj",
    "Bagpat": "Baghpat",
    "Sant Ravidas Nagar": "Bhadohi",
    "Sant Ravidas Nagar Bhadohi": "Bhadohi"
}

for df in all_dfs:
    df["district"] = df["district"].replace(cleanup_map)
    
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

75 75 76


In [12]:
enrol_df["district"] = enrol_df["district"].str.lower()
demo_df["district"] = demo_df["district"].str.lower()
bio_df["district"] = bio_df["district"].str.lower()

In [13]:
print(enrol_df["pincode"].nunique(), demo_df["pincode"].nunique(), bio_df["pincode"].nunique())

1742 1762 1771


In [14]:
# Analyze pincode differences
enrol_pincodes = set(enrol_df["pincode"].dropna())
demo_pincodes = set(demo_df["pincode"].dropna())
bio_pincodes = set(bio_df["pincode"].dropna())

print("Enrollment pincodes:", len(enrol_pincodes))
print("Demographic pincodes:", len(demo_pincodes))
print("Biometric pincodes:", len(bio_pincodes))
print()

# Find differences
only_in_enrol = enrol_pincodes - demo_pincodes - bio_pincodes
only_in_demo = demo_pincodes - enrol_pincodes - bio_pincodes
only_in_bio = bio_pincodes - enrol_pincodes - demo_pincodes
in_all = enrol_pincodes & demo_pincodes & bio_pincodes

print(f"Pincodes only in enrollment: {len(only_in_enrol)}")
print(f"Pincodes only in demographic: {len(only_in_demo)}")
print(f"Pincodes only in biometric: {len(only_in_bio)}")
print(f"Pincodes in all three: {len(in_all)}")
print()

# Check for NULL/NaN values
print("NULL pincodes:")
print(f"Enrollment: {enrol_df["pincode"].isna().sum()}")
print(f"Demographic: {demo_df["pincode"].isna().sum()}")
print(f"Biometric: {bio_df["pincode"].isna().sum()}")
print()

# Show some examples of unique pincodes
print("Sample pincodes only in enrollment:", list(only_in_enrol))
print("Sample pincodes only in demographic:", list(only_in_demo))
print("Sample pincodes only in biometric:", list(only_in_bio))

Enrollment pincodes: 1742
Demographic pincodes: 1762
Biometric pincodes: 1771

Pincodes only in enrollment: 0
Pincodes only in demographic: 3
Pincodes only in biometric: 9
Pincodes in all three: 1739

NULL pincodes:
Enrollment: 0
Demographic: 0
Biometric: 0

Sample pincodes only in enrollment: []
Sample pincodes only in demographic: [204106, 209748, 272141]
Sample pincodes only in biometric: [230145, 229315, 244101, 230123, 272269, 243406, 209746, 221719, 401208]


In [15]:
for df in all_dfs:
    df["month"] = df["date"].dt.month

In [16]:
enrol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131509 entries, 0 to 131508
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            131509 non-null  datetime64[ns]
 1   state           131509 non-null  object        
 2   district        131509 non-null  object        
 3   pincode         131509 non-null  int64         
 4   age_0_5         131509 non-null  int64         
 5   age_5_17        131509 non-null  int64         
 6   age_18_greater  131509 non-null  int64         
 7   month           131509 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(2)
memory usage: 7.5+ MB


In [17]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196621 entries, 0 to 196620
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           196621 non-null  datetime64[ns]
 1   state          196621 non-null  object        
 2   district       196621 non-null  object        
 3   pincode        196621 non-null  int64         
 4   demo_age_5_17  196621 non-null  int64         
 5   demo_age_17_   196621 non-null  int64         
 6   month          196621 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 9.8+ MB


In [18]:
bio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449911 entries, 0 to 449910
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          449911 non-null  datetime64[ns]
 1   state         449911 non-null  object        
 2   district      449911 non-null  object        
 3   pincode       449911 non-null  int64         
 4   bio_age_5_17  449911 non-null  int64         
 5   bio_age_17_   449911 non-null  int64         
 6   month         449911 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 22.3+ MB


In [19]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,month
0,2025-04-01,Uttar Pradesh,maharajganj,273305,141,176,13,4
1,2025-04-01,Uttar Pradesh,banda,210001,250,177,30,4
2,2025-04-01,Uttar Pradesh,lucknow,226005,83,92,23,4
3,2025-04-01,Uttar Pradesh,kanpur nagar,208004,73,44,16,4
4,2025-04-01,Uttar Pradesh,bahraich,271824,68,58,11,4


In [20]:
demo_df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,month
0,2025-03-01,Uttar Pradesh,varanasi,221307,138,1301,3
1,2025-03-01,Uttar Pradesh,budaun,202523,102,1712,3
2,2025-03-01,Uttar Pradesh,fatehpur,212631,77,1221,3
3,2025-03-01,Uttar Pradesh,pilibhit,262203,78,1073,3
4,2025-03-01,Uttar Pradesh,deoria,274208,50,590,3


In [21]:
bio_df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,month
0,2025-04-01,Uttar Pradesh,mau,121705,137,55,4
1,2025-04-01,Uttar Pradesh,hardoi,241121,462,139,4
2,2025-04-01,Uttar Pradesh,kanpur nagar,208008,19,41,4
3,2025-04-01,Uttar Pradesh,azamgarh,224166,42,27,4
4,2025-04-01,Uttar Pradesh,ballia,221717,226,292,4


In [22]:
enrol_agg = enrol_df.groupby(["state", "district", "month"])[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
demo_agg = demo_df.groupby(["state", "district", "month"])[["demo_age_5_17", "demo_age_17_"]].sum().reset_index()
bio_agg = bio_df.groupby(["state", "district", "month"])[["bio_age_5_17", "bio_age_17_"]].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on = ["state", "district", "month"], how = "left").merge(bio_agg, on = ["state", "district", "month"], how = "left")
combined_df.fillna(0, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239


In [23]:
combined_df["E"] = combined_df["age_0_5"] + combined_df["age_5_17"] + combined_df["age_18_greater"]
combined_df["DU"] = combined_df["demo_age_5_17"] + combined_df["demo_age_17_"]
combined_df["BU"] = combined_df["bio_age_5_17"] + combined_df["bio_age_17_"]
combined_df["U"] = combined_df["DU"] + combined_df["BU"]
combined_df["T"] = combined_df["E"] + combined_df["U"]
combined_df.head(20)

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845,906,30907.0,26205,57112.0,58018.0
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368,1437,0.0,24930,24930.0,26367.0
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936,2577,7600.0,41502,49102.0,51679.0
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239,6457,5738.0,26381,32119.0,38576.0
5,Uttar Pradesh,agra,7,2371,2123,75,415.0,3159.0,17526,12151,4569,3574.0,29677,33251.0,37820.0
6,Uttar Pradesh,agra,9,3078,2629,57,5181.0,26084.0,24858,12823,5764,31265.0,37681,68946.0,74710.0
7,Uttar Pradesh,agra,10,1639,1163,61,1739.0,14571.0,15357,11312,2863,16310.0,26669,42979.0,45842.0
8,Uttar Pradesh,agra,11,2638,1771,59,2862.0,38883.0,17642,13208,4468,41745.0,30850,72595.0,77063.0
9,Uttar Pradesh,agra,12,1885,1033,350,4034.0,58377.0,16601,13547,3268,62411.0,30148,92559.0,95827.0


In [24]:
district_monthly_counts = combined_df.groupby(["district", "month"]).agg(total_months = ("month", "count"), active_months = ("T", lambda x : (x > 0).sum())).reset_index()
district_monthly_counts.head()

,district,month,total_months,active_months
0,agra,1,1,1
1,agra,3,1,1
2,agra,4,1,1
3,agra,5,1,1
4,agra,6,1,1


In [25]:
district_monthly_counts["zero_months"] = district_monthly_counts["total_months"] - district_monthly_counts["active_months"]
district_monthly_counts["activity_ratio"] = district_monthly_counts["active_months"] / district_monthly_counts["total_months"]
district_monthly_counts["zero_month_ratio"] = district_monthly_counts["zero_months"] / district_monthly_counts["total_months"]

combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845,906,30907.0,26205,57112.0,58018.0,1.0,0.0
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368,1437,0.0,24930,24930.0,26367.0,1.0,0.0
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936,2577,7600.0,41502,49102.0,51679.0,1.0,0.0
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239,6457,5738.0,26381,32119.0,38576.0,1.0,0.0


In [26]:
combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,1.0,0.0
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845,906,30907.0,26205,57112.0,58018.0,1.0,0.0,1.0,0.0
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368,1437,0.0,24930,24930.0,26367.0,1.0,0.0,1.0,0.0
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936,2577,7600.0,41502,49102.0,51679.0,1.0,0.0,1.0,0.0
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239,6457,5738.0,26381,32119.0,38576.0,1.0,0.0,1.0,0.0


In [27]:
district_volume_metrics = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrolment = ("E", "mean"),
                                               monthly_valatility = ("T", lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
                                               peak_load_ratio = ("T", lambda x: x.max() / x.mean() if x.mean() > 0 else 0)).reset_index()


combined_df = combined_df.merge(district_volume_metrics, on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y,avg_monthly_enrolment,monthly_valatility,peak_load_ratio
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845,906,30907.0,26205,57112.0,58018.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368,1437,0.0,24930,24930.0,26367.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936,2577,7600.0,41502,49102.0,51679.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239,6457,5738.0,26381,32119.0,38576.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742


In [28]:
district_update_burden = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrollments = ("E", "sum"), avg_monthly_demo_updates = ("DU", "sum"), avg_monthly_bio_updates = ("BU", "sum")).reset_index()

district_update_burden["U"] = district_update_burden["avg_monthly_demo_updates"] + district_update_burden["avg_monthly_bio_updates"]
district_update_burden["biometric_burden"] = district_update_burden["avg_monthly_bio_updates"] / (district_update_burden["avg_monthly_bio_updates"] + district_update_burden["avg_monthly_demo_updates"])
district_update_burden["update_dominant"] = np.where(district_update_burden["U"] > district_update_burden["avg_monthly_enrollments"], 1, 0)
district_update_burden["enrollment_update_balance"] = district_update_burden["avg_monthly_enrollments"] / (district_update_burden["avg_monthly_enrollments"] + district_update_burden["U"])

combined_df = combined_df.merge(district_update_burden[["state", "district", "biometric_burden", "update_dominant", "enrollment_update_balance"]], on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742,0.579059,1,0.06519
1,Uttar Pradesh,agra,3,280,494,132,2736.0,28171.0,16360,9845,906,30907.0,26205,57112.0,58018.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742,0.579059,1,0.06519
2,Uttar Pradesh,agra,4,832,568,37,0.0,0.0,17562,7368,1437,0.0,24930,24930.0,26367.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742,0.579059,1,0.06519
3,Uttar Pradesh,agra,5,1550,967,60,922.0,6678.0,26566,14936,2577,7600.0,41502,49102.0,51679.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742,0.579059,1,0.06519
4,Uttar Pradesh,agra,6,2971,3265,221,528.0,5210.0,15142,11239,6457,5738.0,26381,32119.0,38576.0,1.0,0.0,1.0,0.0,3337.5,0.489516,1.871742,0.579059,1,0.06519


In [29]:
combined_df.drop(["activity_ratio_y", "zero_month_ratio_y"], axis=1, inplace=True)
combined_df.rename(columns={"activity_ratio_x": "activity_ratio", "zero_month_ratio_x": "zero_month_ratio"}, inplace=True)

In [30]:
combined_df = combined_df.groupby(["state", "district"], as_index = False).first()
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,3337.500000,0.489516,1.871742,0.579059,1,0.065190
1,Uttar Pradesh,aligarh,1,537,372,20,135.0,1937.0,1043,913,929,2072.0,1956,4028.0,4957.0,1.0,0.0,2909.500000,0.416973,1.635248,0.542271,1,0.061194
2,Uttar Pradesh,ambedkar nagar,1,235,263,2,89.0,891.0,640,570,500,980.0,1210,2190.0,2690.0,1.0,0.0,1360.571429,0.602735,1.828178,0.556144,1,0.062670
3,Uttar Pradesh,amethi,1,271,215,4,46.0,816.0,643,766,490,862.0,1409,2271.0,2761.0,1.0,0.0,1428.166667,0.522173,1.610576,0.517258,1,0.061917
4,Uttar Pradesh,amroha,1,404,245,14,70.0,631.0,914,834,663,701.0,1748,2449.0,3112.0,1.0,0.0,1405.000000,0.562596,1.781120,0.527589,1,0.062551


In [31]:
def normalize(x):
    maxx, minx = x.max(), x.min()
    if maxx == minx:
        return x * 0 + 0.5
    normalized = (x - minx) / (maxx - minx)
    return normalized

def inverse_normalize(x):
    inversed = 1 - normalize(x)
    return inversed

In [32]:
combined_df["access"] = (combined_df["activity_ratio"] + normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["responsiveness"] = normalize(combined_df["U"] / (combined_df["E"] + combined_df["U"]))
combined_df["inclusion"] = normalize((combined_df["age_0_5"] + combined_df["age_5_17"]) / combined_df["E"])
combined_df["stability"] = (inverse_normalize(combined_df["monthly_valatility"]) + inverse_normalize(combined_df["peak_load_ratio"])) / 2
combined_df["visibility"] = combined_df["activity_ratio"]

combined_df["DEI"] = (combined_df["access"] + combined_df["responsiveness"] + combined_df["inclusion"] + combined_df["stability"] + combined_df["visibility"]) / 5
combined_df["ASS"] = (inverse_normalize(combined_df["activity_ratio"]) + inverse_normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["UBS"] = (normalize(combined_df["biometric_burden"]) + normalize(combined_df["update_dominant"])) / 2
combined_df["SRS"] = (normalize(combined_df["monthly_valatility"]) + normalize(combined_df["zero_month_ratio"])) / 2

combined_df.head()


,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,access,responsiveness,inclusion,stability,visibility,DEI,ASS,UBS,SRS
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,3337.500000,0.489516,1.871742,0.579059,1,0.065190,0.885522,0.519635,0.858630,0.536612,1.0,0.760080,0.364478,0.562597,0.418064
1,Uttar Pradesh,aligarh,1,537,372,20,135.0,1937.0,1043,913,929,2072.0,1956,4028.0,4957.0,1.0,0.0,2909.500000,0.416973,1.635248,0.542271,1,0.061194,0.825100,0.435664,0.705059,0.813286,1.0,0.755822,0.424900,0.494328,0.281395
2,Uttar Pradesh,ambedkar nagar,1,235,263,2,89.0,891.0,640,570,500,980.0,1210,2190.0,2690.0,1.0,0.0,1360.571429,0.602735,1.828178,0.556144,1,0.062670,0.606432,0.446751,0.945200,0.349100,1.0,0.669497,0.643568,0.520072,0.631366
3,Uttar Pradesh,amethi,1,271,215,4,46.0,816.0,643,766,490,862.0,1409,2271.0,2761.0,1.0,0.0,1428.166667,0.522173,1.610576,0.517258,1,0.061917,0.615974,0.507313,0.888163,0.629698,1.0,0.728230,0.634026,0.447911,0.479590
4,Uttar Pradesh,amroha,1,404,245,14,70.0,631.0,914,834,663,701.0,1748,2449.0,3112.0,1.0,0.0,1405.000000,0.562596,1.781120,0.527589,1,0.062551,0.612704,0.250883,0.710709,0.452579,1.0,0.605375,0.637296,0.467082,0.555746


In [33]:
combined_df.drop(["access", "responsiveness", "inclusion", "stability", "visibility"], axis=1, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,DEI,ASS,UBS,SRS
0,Uttar Pradesh,agra,1,567,488,11,173.0,1736.0,1693,1397,1066,1909.0,3090,4999.0,6065.0,1.0,0.0,3337.500000,0.489516,1.871742,0.579059,1,0.065190,0.760080,0.364478,0.562597,0.418064
1,Uttar Pradesh,aligarh,1,537,372,20,135.0,1937.0,1043,913,929,2072.0,1956,4028.0,4957.0,1.0,0.0,2909.500000,0.416973,1.635248,0.542271,1,0.061194,0.755822,0.424900,0.494328,0.281395
2,Uttar Pradesh,ambedkar nagar,1,235,263,2,89.0,891.0,640,570,500,980.0,1210,2190.0,2690.0,1.0,0.0,1360.571429,0.602735,1.828178,0.556144,1,0.062670,0.669497,0.643568,0.520072,0.631366
3,Uttar Pradesh,amethi,1,271,215,4,46.0,816.0,643,766,490,862.0,1409,2271.0,2761.0,1.0,0.0,1428.166667,0.522173,1.610576,0.517258,1,0.061917,0.728230,0.634026,0.447911,0.479590
4,Uttar Pradesh,amroha,1,404,245,14,70.0,631.0,914,834,663,701.0,1748,2449.0,3112.0,1.0,0.0,1405.000000,0.562596,1.781120,0.527589,1,0.062551,0.605375,0.637296,0.467082,0.555746


In [ ]:
combined_df.to_csv(r"D:\UIDAI hackathon\North\UP\up_district_analysis.csv", index=False)
final_df = combined_df[["state", "district", "DEI", "ASS", "UBS", "SRS"]]
final_df.to_csv(r"D:\UIDAI hackathon\North\UP\up_district_final_scores.csv", index=False)